In [ ]:
function [V,F,I_FD] = spatialDecomposition(X,unitCell)
% decomposite the spatial domain into cells D with vertices V,
%
% Output
%  V - list of vertices
%  F - list of faces
%  I_FD - incidence matrix between faces to cells

% compute voronoi decomposition
% V - list of vertices of the Voronoi cells
% D   - cell array of Vornoi cells with centers X_D ordered accordingly
if isempty(unitCell), unitCell = calcUnitCell(X); end
if check_option(varargin,'unitCell')
  
  % compute the vertices
  [V,faces] = generateUnitCells(X,unitCell,varargin{:});
 
  D = cell(size(X,1),1);
  for k=1:size(X,1)  
    D{k} = faces(k,:);
  end
  
else
  
  dummyCoordinates = calcBoundary(X,unitCell,varargin{:});

  [V,D] = voronoin([X;dummyCoordinates],{'Q5','Q6','Qs'}); %,'QbB'

  D = D(1:size(X,1));
  
end

% now we need some adjacencies and incidences
iv = [D{:}];            % nodes incident to cells D
id = zeros(size(iv));   % number the cells
    
p = [0; cumsum(cellfun('prodofsize',D))];
for k=1:numel(D), id(p(k)+1:p(k+1)) = k; end
    
% next vertex
indx = 2:numel(iv)+1;
indx(p(2:end)) = p(1:end-1)+1;
ivn = iv(indx);

% edges list
F = [iv(:), ivn(:)];

% should be unique (i.e one edge is incident to two cells D)
[F, ~, ie] = unique(sort(F,2),'rows');

% faces incident to cells, F x D
I_FD = sparse(ie,id,1);

end


# numpy.loadtxt
function dummyCoordinates = calcBoundary(X,unitCell,varargin)
def calc_boundary(X, unit_cell, var_arg_in):
    '''
    dummy coordinates so that the voronoi-cells of X are finite

    Inputs:
    --------------
    X : n x 2 numpy array of [x,y] vertex coordinates

    unit_cell : n x 2 numpy array of [x,y] coordinates of "pixel" boundary (e.g. hexagon or square)

    var_arg_in : ???
    ???


    Outputs:
    --------------
    ??? : ???
    ???

    Dependencies:
    --------------
    import getopt
    import numpy as np
    from scipy.spatial import ConvexHull
    from numpy import arctan2, diff, cumsum, matrix, squeeze, unique
    from statistics import mean
    from math import sqrt, floor, copysign
    '''
    # Imports
    import getopt
    import numpy as np
    from scipy.spatial import ConvexHull
    from numpy import arctan2, diff, cumsum, matrix, squeeze, unique
    from statistics import mean
    from math import sqrt, floor, copysign

    dummy_coordinates = []

    # specify a bounding polyogn
    boundary = getopt(var_arg_in,'boundary','hull',{'char','double'})

    if boundary.isalpha():

    if boundary.lower() == 'hull' \
        or boundary.lower() == 'convexhull': 
        x = X[:,1]
        y = X[:,2]

        k = ConvexHull(x,y)
        
        # erase all linearly dependent points
        angle = arctan2( 
            x[k[1:-2]]-x[k[2:]],
            y[k[1:-2]]-y[k[2:]] )
        
        # ######################################## # get_linearly_dependent_points()
        k = k([true; abs(diff(angle))>eps; true]);
        # ######################################## # 
        bounding_X = X[k, :]
        
    elif 'cube':
        # set up a rectangular box
        envelope_X = [min(X), max(X)]
        bounding_X = [
            envelope_X[1], envelope_X[3],
            envelope_X[2], envelope_X[3],
            envelope_X[2], envelope_X[4],
            envelope_X[1], envelope_X[4],
            envelope_X[1], envelope_X[3],
            ]

    else:
        raise ValueError('Unknown boundary type. Available options are \
           ''convexhull'' and ''cube''.')

elif isinstance(boundary, float):
    bounding_X = boundary

radius = mean( sqrt( sum(unit_cell**2, axis = 1) ) )
edge_length = sqrt( sum( diff(bounding_X)**2, axis = 1) )

# fill each line segment with nodes every 20 points (in average)
nto = floor( (edge_length>0)*4 )
nto = floor( edge_length*(2*radius) )

cs = cumsum([1; 1 + nto]);
bounding_X[cs, :] = bounding_X

# interpolation
for k in range(len(nto)):
    for dim in range(2):
        bounding_X(cs[k]:cs[k+1], dim) = linspace[
            bounding_X(cs[k],dim),
            bounding_X(cs[k+1], dim),
            nto(k)+2
        ]

# homogeneous coordinates
X[:, 3] = 1
bounding_X[:, 3] = 1

################################################################################
#------------------------------------------------------------------------------#
#------------------------------------------------------------------------------#
                        # REQUIRES MATRIX INSTANTIATION #
#------------------------------------------------------------------------------#
#------------------------------------------------------------------------------#
#------------------------------------------------------------------------------#

# householder matrix
H = @(v) eye(3) - 2./(v(:)'*v(:))*(v(:)*v(:)')
# translation matrix
T = matrix[ 1 0 s(1); 0 1 s(2); 0 0 1]

#------------------------------------------------------------------------------#
#------------------------------------------------------------------------------#
#------------------------------------------------------------------------------#
                        # REQUIRES MATRIX INSTANTIATION #
#------------------------------------------------------------------------------#
#------------------------------------------------------------------------------#
################################################################################

# direction of the edge
edge_direction = diff(bounding_X)
edge_angle = arctan2( edge_direction[:, 2], edge_direction[:,1] )
edge_length = sqrt( sum(edge_direction**2, axis = 1) )

################################################################################
#------------------------------------------------------------------------------#
#------------------------------------------------------------------------------#
                        # REQUIRES SUBROUTINE CONSTRUCTION #
#------------------------------------------------------------------------------#
#------------------------------------------------------------------------------#
#------------------------------------------------------------------------------#

# shift the starting vertex
b_X = squeeze( float( axis_2_quat(z_vector, edge_angle) \
    vector3d([0; radius; 1]) ) )
offset_X = b_X - bounding_X[1:end-1,:] ## ??????????????????????????????????????

#------------------------------------------------------------------------------#
#------------------------------------------------------------------------------#
#------------------------------------------------------------------------------#
                        # REQUIRES SUBROUTINE CONSTRUCTION #
#------------------------------------------------------------------------------#
#------------------------------------------------------------------------------#
################################################################################

for k in range(bounding_X.shape[0]-1):
    
    # mirror the point set X on each edge
    p_X = X * -1*( T[offset_X[k, :]] * H[edge_direction[k, :]] * T[offset_X[k, :]] ).T
    
    # distance between original and mirrored point
    dist = sqrt( sum( (X[:,1:2]-p_X[:,1:2])**2, axis = 1 ) )

    intend_X = 2 * radius * copysign(1, edge_direction[k,1:2])
    
    # now try to delete unnecessary points
    m = 2

    condition1 = True
    while condition1:
    
        tmp_X = p_X[dist < m*radius,1:2]
        
################################################################################
#------------------------------------------------------------------------------#
#------------------------------------------------------------------------------#
                        # REQUIRES SUBROUTINE CONSTRUCTION #
#------------------------------------------------------------------------------#
#------------------------------------------------------------------------------#
#------------------------------------------------------------------------------#

        right = [bsxfun(@minus, tmpX, boundingX(k,1:2)  - intendX ) * edgeDirection(k,1:2).T] < 0
        left  = [bsxfun(@minus, tmpX, boundingX(k+1,1:2)+ intendX ) * edgeDirection(k,1:2).T] > 0

#------------------------------------------------------------------------------#
#------------------------------------------------------------------------------#
#------------------------------------------------------------------------------#
                        # REQUIRES SUBROUTINE CONSTRUCTION #
#------------------------------------------------------------------------------#
#------------------------------------------------------------------------------#
################################################################################
        
        tmp_X = tmp_X[ not(right or left), :]
        
        if edge_length(k) / tmp_X.shape[0] < radius/3:
            break
        elif m < 2**7:
            m = m*2
        else:
            m = m+10
    
    dummy_coordinates = np.array([[dummy_coordinates, tmp_X]]).T
    
_, dummy_coordinates, _, _ = unique(dummyCoordinates, axis = 0)



In [ ]:
#from pyhull.convex_hull import ConvexHull
def erase_linearly_dependent_points(points):
    '''
    subfunction to remove linearly dependent points.

    Inputs:
    --------------
    k : ???
        ???

    Outputs:
    --------------
    ??? : ???
        ???

    Dependencies:
    --------------
    from scipy.spatial import ConvexHull
    '''
    from scipy.spatial import ConvexHull
    k = ConvexHull(points)

    # erase all linear dependent points
    angle = np.arctan2( 
        x[k.vertices[0:-1]]-x[k.vertices[1:]],
        y[k.vertices[0:-1]]-y[k.vertices[1:]]
    )
    test = np.abs(np.diff(angle))>np.spacing(1.0)
    k2 = k.vertices[np.concatenate([[True], test, [True]])]
    boundingX = [x[k2], y[k2]]

    cellRot=0

################################################################################
#------------------------------------------------------------------------------#
#------------------------------------------------------------------------------#
                        # REQUIRES SUBROUTINE CONSTRUCTION #
#------------------------------------------------------------------------------#
#------------------------------------------------------------------------------#
#------------------------------------------------------------------------------#
    
    unitCell = util.regularPoly(4,xstp,cellRot)

#------------------------------------------------------------------------------#
#------------------------------------------------------------------------------#
#------------------------------------------------------------------------------#
                        # REQUIRES SUBROUTINE CONSTRUCTION #
#------------------------------------------------------------------------------#
#------------------------------------------------------------------------------#
################################################################################

    print(unitCell)
    radius = np.mean( np.sqrt(np.sum(unitCell**2,2)) )
    edgeLength = np.sqrt( np.sum( np.diff(boundingX)**2, axis = 2) )

In [13]:
import numpy as np
print((np.array([[2, 1]])).T)

[[2]
 [1]]
